In [1]:
import pandas as pd
data_filename = "basketball.csv"
dataset = pd.read_csv(data_filename)

In [2]:
#conda install panda

In [3]:
dataset.head(5)

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,"Tue, Oct 29, 2013",7:00p,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN
1,"Tue, Oct 29, 2013",8:00p,Chicago Bulls,95,Miami Heat,107,Box Score,NaN
2,"Tue, Oct 29, 2013",10:30p,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN
3,"Wed, Oct 30, 2013",7:00p,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN
4,"Wed, Oct 30, 2013",7:00p,Boston Celtics,87,Toronto Raptors,93,Box Score,NaN


In [4]:
dataset = pd.read_csv(data_filename, parse_dates=["Date"])
dataset.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Notes"]
# dataset = dataset.drop("Unnamed: 8", axis=1)
# dataset.columns = ['Date', 'ScoreType', 'VisitorTeam', 'VisitorPts', 'HomeTeam', 'HomePts', 'OT?', 'Notes']

In [5]:
dataset.head()

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,2013-10-29,7:00p,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN
1,2013-10-29,8:00p,Chicago Bulls,95,Miami Heat,107,Box Score,NaN
2,2013-10-29,10:30p,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN
3,2013-10-30,7:00p,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN
4,2013-10-30,7:00p,Boston Celtics,87,Toronto Raptors,93,Box Score,NaN


In [6]:
print(dataset.dtypes)

Date            datetime64[ns]
Start (ET)              object
Visitor Team            object
VisitorPts               int64
Home Team               object
HomePts                  int64
OT?                     object
Notes                   object
dtype: object


In [7]:
dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]

In [8]:
y_true = dataset["HomeWin"].values

In [9]:
dataset["HomeWin"].mean()

0.579226686884003

In [10]:
from collections import defaultdict
won_last = defaultdict(int)

In [11]:
dataset["HomeLastWin"] = 0
dataset["VisitorLastWin"] = 0

In [12]:
# for index, row in dataset.iterrows():
#     home_team = row["HomeTeam"]
#     visitor_team = row["VisitorTeam"]
#     row["HomeLastWin"] = won_last[home_team]
#     row["VisitorLastWin"] = won_last[visitor_team]
#     won_last[home_team] = row["HomeWin"]
#     won_last[visitor_team] = not row["HomeWin"]
#     dataset.loc[index, "HomeLastWin"] = row["HomeLastWin"]
#     dataset.loc[index, "VisitorLastWin"] = row["VisitorLastWin"]
    

In [13]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    dataset.iloc[index] = row
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]


In [14]:
dataset.iloc[20:25]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
20,2013-11-01,7:00p,New Orleans Pelicans,90,Orlando Magic,110,Box Score,NaN,True,False,False
21,2013-11-01,7:00p,Philadelphia 76ers,109,Washington Wizards,102,Box Score,NaN,False,False,True
22,2013-11-01,7:30p,Toronto Raptors,95,Atlanta Hawks,102,Box Score,NaN,True,False,True
23,2013-11-01,7:30p,Milwaukee Bucks,105,Boston Celtics,98,Box Score,NaN,False,False,False
24,2013-11-01,8:00p,Miami Heat,100,Brooklyn Nets,101,Box Score,NaN,True,False,False


In [15]:
print(dataset.dtypes)

Date              datetime64[ns]
Start (ET)                object
Visitor Team              object
VisitorPts                 int64
Home Team                 object
HomePts                    int64
OT?                       object
Notes                     object
HomeWin                     bool
HomeLastWin               object
VisitorLastWin            object
dtype: object


In [16]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [17]:
X_previouswins = dataset[["HomeLastWin", "VisitorLastWin"]].values

In [18]:
from sklearn.model_selection import cross_val_score
import numpy as np
scores = cross_val_score(clf, X_previouswins, y_true,
scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 56.4%


In [19]:
import os
standings_filename = os.path.join( "standing.csv")
standings = pd.read_csv(standings_filename, skiprows=0)

In [20]:
standings

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,Miami Heat,66-16,Apr-37,12月29日,Nov-41,5月25日,4月14日,12月6日,1月15日,...,Feb-30,9月3日,Aug-39,1-0,10月3日,10月5日,8月5日,12月1日,1月17日,8月1日
1,2,Oklahoma City Thunder,60-22,Jul-34,26-15,9月21日,39-13,7月3日,8月2日,6月4日,...,8月21日,3月6日,Jun-44,NaN,4月13日,11月2日,11月5日,7月4日,12月5日,6月2日
2,3,San Antonio Spurs,58-24,Jun-35,23-18,5月25日,33-19,8月2日,9月1日,8月2日,...,12月16日,9月5日,10月31日,1-0,12月4日,12月4日,12月3日,8月3日,10月4日,3月6日
3,4,Denver Nuggets,57-25,Mar-38,19-22,11月19日,38-14,5月5日,10-0,4月6日,...,4月24日,11月7日,8月28日,0-1,8月8日,9月6日,12月3日,8月4日,2月13日,7月1日
4,5,Los Angeles Clippers,56-26,Sep-32,24-17,9月21日,35-17,7月3日,8月2日,6月4日,...,9月17日,3月5日,Dec-38,1-0,8月6日,16-0,9月7日,8月5日,7月7日,7月1日
5,6,Memphis Grizzlies,56-26,Sep-32,24-17,8月22日,34-18,8月2日,8月2日,6月4日,...,8月23日,6月4日,9月28日,0-1,12月1日,7月7日,10月7日,9月2日,11月6日,7月2日
6,7,New York Knicks,54-28,10月31日,23-18,37-15,17-13,10月6日,12月6日,3月15日,...,10月22日,7月5日,12月31日,NaN,11月4日,10月5日,7月6日,6月5日,12月6日,8月2日
7,8,Brooklyn Nets,49-33,26-15,23-18,36-16,13-17,11月5日,5月13日,12月6日,...,11月18日,9月4日,23-17,NaN,11月4日,5月11日,11月4日,7月5日,8月7日,7月2日
8,9,Indiana Pacers,49-32,11月30日,19-21,31-20,12月18日,6月11日,3月13日,12月6日,...,11月17日,4月9日,27-14,1-0,7月8日,10月5日,9月6日,9月3日,11月5日,2月5日
9,10,Golden State Warriors,47-35,28-13,19-22,11月19日,28-24,7月3日,5月5日,7月3日,...,17-13,5月3日,20-18,1-0,8月6日,12月4日,8月7日,4月8日,9月7日,5月3日


In [21]:
dataset["HomeTeamRanksHigher"] = 0
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    
    if home_team == "New Orleans Pelicans":
        home_team = "New Orleans Hornets"
    elif visitor_team == "New Orleans Pelicans":
        visitor_team = "New Orleans Hornets"
    home_rank = standings[standings["Team"] == home_team]["Rk"].values[0]
    visitor_rank = standings[standings["Team"] == visitor_team]["Rk"].values[0]
    row["HomeTeamRanksHigher"] = int(home_rank > visitor_rank)
    dataset.iloc[index] = row


In [22]:
X_homehigher = dataset[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher"]].values

In [23]:
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 60.0%


In [24]:
last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0

In [25]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    dataset.iloc[index] = row
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner

In [26]:
X_lastwinner = dataset[["HomeTeamRanksHigher", "HomeTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_lastwinner, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 59.9%


In [27]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()


In [28]:
encoding.fit(dataset["Home Team"].values)

LabelEncoder()

In [29]:
home_teams = encoding.transform(dataset["Home Team"].values)
visitor_teams = encoding.transform(dataset["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

In [30]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

In [31]:
X_teams_expanded = onehot.fit_transform(X_teams).todense()


In [32]:
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams_expanded, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 60.5%


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\user\anaconda3\lib\site-pack

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 58.0%


In [34]:
X_all = np.hstack([X_homehigher, X_teams])
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))


Accuracy: 61.0%


In [35]:
from sklearn.model_selection import GridSearchCV
parameter_space = {
 "max_features": [2, 10, 'auto'],
 "n_estimators": [100,],
 "criterion": ["gini", "entropy"],
 "min_samples_leaf": [2, 4, 6],
}
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\

Accuracy: 61.7%


In [36]:
print(grid.best_estimator_)


RandomForestClassifier(criterion='entropy', max_features=2, min_samples_leaf=4,
                       random_state=14)


# 距離上次比賽有多少天：

In [37]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['DaysSinceLastGame'] = 0

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    date = row["Date"]

    home_last_game = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == home_team) | (dataset['Visitor Team'] == home_team))].sort_values(by='Date', ascending=False).head(1)
    visitor_last_game = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == visitor_team) | (dataset['Visitor Team'] == visitor_team))].sort_values(by='Date', ascending=False).head(1)

    if not home_last_game.empty:
        dataset.at[index, 'DaysSinceLastGame'] = (date - home_last_game['Date'].values[0]).days

    if not visitor_last_game.empty:
        dataset.at[index, 'DaysSinceLastGame'] = max((date - visitor_last_game['Date'].values[0]).days, dataset.at[index, 'DaysSinceLastGame'])


# 最近五場比賽贏了幾場：

In [38]:
dataset['Last5GamesWins'] = 0

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    date = row["Date"]

    home_last_5_games = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == home_team) | (dataset['Visitor Team'] == home_team))].sort_values(by='Date', ascending=False).head(5)
    visitor_last_5_games = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == visitor_team) | (dataset['Visitor Team'] == visitor_team))].sort_values(by='Date', ascending=False).head(5)

    home_wins = sum(home_last_5_games["HomeWin"][home_last_5_games["Home Team"] == home_team]) + sum(~home_last_5_games["HomeWin"][home_last_5_games["Visitor Team"] == home_team])
    visitor_wins = sum(visitor_last_5_games["HomeWin"][visitor_last_5_games["Home Team"] == visitor_team]) + sum(~visitor_last_5_games["HomeWin"][visitor_last_5_games["Visitor Team"] == visitor_team])

    dataset.at[index, 'Last5GamesWins'] = home_wins - visitor_wins


In [39]:
dataset['Last5GamesWins'] = 0

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    date = row["Date"]

    home_last_5_games = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == home_team) | (dataset['Visitor Team'] == home_team))].sort_values(by='Date', ascending=False).head(5)
    visitor_last_5_games = dataset[(dataset['Date'] < date) & ((dataset['Home Team'] == visitor_team) | (dataset['Visitor Team'] == visitor_team))].sort_values(by='Date', ascending=False).head(5)

    home_wins = sum(home_last_5_games["HomeWin"][home_last_5_games["Home Team"] == home_team]) + sum(~home_last_5_games["HomeWin"][home_last_5_games["Visitor Team"] == home_team])
    visitor_wins = sum(visitor_last_5_games["HomeWin"][visitor_last_5_games["Home Team"] == visitor_team]) + sum(~visitor_last_5_games["HomeWin"][visitor_last_5_games["Visitor Team"] == visitor_team])

    dataset.at[index, 'Last5GamesWins'] = home_wins - visitor_wins


# 已經為數據集添加了新的特徵，然後使用隨機森林和GridSearchCV重新訓練模型：

In [40]:
X_new_features = dataset[["DaysSinceLastGame", "Last5GamesWins"]].values
X_all = np.hstack([X_all, X_new_features])

clf = RandomForestClassifier(random_state=14)
parameter_space = {
 "max_features": [2, 10, 'auto'],
 "n_estimators": [100,],
 "criterion": ["gini", "entropy"],
 "min_samples_leaf": [2, 4, 6],
}
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)


Accuracy: 60.8%
RandomForestClassifier(criterion='entropy', max_features=2, min_samples_leaf=6,
                       random_state=14)


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\

# 隊伍是否在客場與特定隊伍交手會取得好成績，然後使用隨機森林和GridSearchCV重新訓練模型：

In [41]:
def visitor_team_performance(row, dataset):
    visitor_team = row["Visitor Team"]
    home_team = row["Home Team"]
    total_games = len(dataset[((dataset["Visitor Team"] == visitor_team) & (dataset["Home Team"] == home_team))])
    total_wins = len(dataset[((dataset["Visitor Team"] == visitor_team) & (dataset["Home Team"] == home_team) & (~dataset["HomeWin"]))])

    if total_games == 0:
        return 0
    else:
        return total_wins / total_games

dataset["VisitorTeamGoodPerformance"] = dataset.apply(lambda row: visitor_team_performance(row, dataset), axis=1)


In [42]:
X_new_feature = dataset[["VisitorTeamGoodPerformance"]].values
X_all = np.hstack([X_all, X_new_feature])

clf = RandomForestClassifier(random_state=14)
parameter_space = {
 "max_features": [2, 10, 'auto'],
 "n_estimators": [100,],
 "criterion": ["gini", "entropy"],
 "min_samples_leaf": [2, 4, 6],
}
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)


Accuracy: 82.0%
RandomForestClassifier(criterion='entropy', max_features=2, min_samples_leaf=4,
                       random_state=14)


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\

# 單純跑課本的程式碼，正確率大約為60.4%，試著增加距離上次比賽的天數，以及最近五場比賽的勝場數，準確率會提升到61.4%，推斷可能這兩個特徵影響不大，可能原因，每支球隊都有體補球員，加上運動員身體素質很好，可以很好的調適，不受最近勝敗影響，因此提升不大。後來加入是否在客場與特定隊伍交手會取得好成績，意外提升準確率83.8%，可能每支球隊之間有相剋關係，因此遇到相同隊伍，可以用以往成績來盼這次的輸贏。